In [1]:
import git
import pandas as pd

In [2]:
repo = git.Repo("../")
tree = repo.tree()

commit_dates = pd.DataFrame(columns=['filename', 'first_commit', 'latest_commit'])

for blob in tree.traverse():
    if (blob.path.startswith('submissions') & blob.path.endswith('.csv')):
        commits = list(repo.iter_commits(paths=blob.path))
        commit_dates.loc[len(commit_dates)] = [blob.path.split("/")[-1], # filename
                                               str(pd.to_datetime(commits[-1].committed_date, unit='s').date()), # first commit
                                               str(pd.to_datetime(commits[0].committed_date, unit='s').date())] # latest commit

In [3]:
commit_dates[["forecast_date", "source", "indicator", "model"]] = commit_dates["filename"].str.extract(
    r"^(\d{4}-\d{2}-\d{2})-([^-]+)-([^-]+)-([^.]+)"
)

In [4]:
commit_dates['forecast_date'] = pd.to_datetime(commit_dates.forecast_date)

commit_dates['latest_commit'] = pd.to_datetime(commit_dates.latest_commit)

commit_dates['diff'] = commit_dates.latest_commit - commit_dates.forecast_date

commit_dates['retrospective'] = commit_dates['diff'] > pd.Timedelta(days=3)

In [5]:
commit_dates.head()

,filename,first_commit,latest_commit,forecast_date,source,indicator,model,diff,retrospective
0,2024-10-17-agi-are-KIT-LightGBM.csv,2024-11-07,2025-04-24,2024-10-17,agi,are,KIT-LightGBM,189 days,True
1,2024-10-24-agi-are-KIT-LightGBM.csv,2024-11-06,2025-04-24,2024-10-24,agi,are,KIT-LightGBM,182 days,True
2,2024-10-31-agi-are-KIT-LightGBM.csv,2024-11-06,2025-04-24,2024-10-31,agi,are,KIT-LightGBM,175 days,True
3,2024-11-07-agi-are-KIT-LightGBM.csv,2024-11-08,2025-04-24,2024-11-07,agi,are,KIT-LightGBM,168 days,True
4,2024-11-14-agi-are-KIT-LightGBM.csv,2024-11-22,2025-04-24,2024-11-14,agi,are,KIT-LightGBM,161 days,True


In [7]:
commit_dates.to_csv('../respinow_viz/plot_data/other/list_commit_dates.csv', index=False)